# GPT-3.5-Turbo on GSM8K

In [1]:
!pip list

Package                   Version
------------------------- --------------
aiohttp                   3.9.5
aiosignal                 1.3.1
annotated-types           0.7.0
anyio                     4.4.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
async-timeout             4.0.3
atlassian-python-api      3.41.13
attrs                     23.2.0
Babel                     2.15.0
backcall                  0.2.0
beautifulsoup4            4.12.3
bleach                    6.1.0
certifi                   2024.6.2
cffi                      1.16.0
charset-normalizer        3.3.2
comm                      0.2.2
datasets                  2.20.0
debugpy                   1.8.1
decorator                 5.1.1
defusedxml                0.7.1
Deprecated                1.2.14
dill                      0.3.8
distro                    1.9.0
exceptiongroup  

In [2]:
!pip install openai;

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install tenacity

Defaulting to user installation because normal site-packages is not writeable


In [6]:
# 아래 프로젝트 환경에서 실행
# 1. source ${project_path}/venv_path/bin/activate 
# 2. (venv_examples)  ..(project path, user name).. pip uninstall urllib3 
# 3. pip install 'urllib3<2.0'
# 4. pip list | grep -i urllib3

In [7]:
import openai
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [8]:
openai.api_key = ""

In [9]:
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('./validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

In [10]:
gsm8k['train'][0]['question']

'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [11]:
gsm8k_test = gsm8k['test']

In [12]:
prompt_complex = open('./prompt_hardest.txt').read()

In [13]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [14]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    return openai.chat.completions.create(**kwargs)

In [15]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)
                
    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [24]:
question = gsm8k_test[1]['question']

In [25]:
prompt_q = prompt_complex + '\nQuestion: ' + question + '\n'

In [26]:
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [18]:
response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q},
    ]
)

In [20]:
print(response.choices[0].message.content)

To find how many bolts in total are needed for the robe, we need to sum the bolts of blue fiber and half the amount of white fiber needed.

Given:
- 2 bolts of blue fiber
- Half that much white fiber, which is 2 / 2 = 1 bolt

Therefore, the total number of bolts needed for the robe is 2 bolts + 1 bolt = 3 bolts. 

The answer is 3 bolts.


<h1> LLMLinugua - Compress Prompt </h1>

In [23]:
!pip install llmlingua

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 6.2 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.9 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 MB 6.8 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.3 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 5.3 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 3.6 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 5.5 MB/s eta 0:00:000m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 9.3 MB/s eta 0:00:000m eta 0:

In [34]:
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [38]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(device_map="mps")
compressed_prompt = llm_lingua.compress_prompt(prompt_complex, target_token=300)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
print(compressed_prompt)

{'compressed_prompt': ': Angelo andanie to plan many hours study their test2 of4 to They should study study total week they1 hour,3 breaks and\ns think step\nanie eachters hours hours\nate1 hours each hours\n andanie to start with planning study,\n to. they to203ute\nunch for  minutes hour3\nSo andanie study study,5 to for\nThe4\n\n: Marks team52.  score teams\ns,5\n team also63, they8\n and as point\n together0\ns scoreds scored4 half57\nThe is The\n: B has two as than deck she2 more what of have she\n\n haveella manybles as0\n Bella25 will\n more deck, have\n2+0The\n4 fruitanas first2 many there\n\n\n,8\nets are3anges theThe3 basket4\nThe1\n:13ruits between cost\n\n, then62\n1If\n of is and price\n can)2\n\n:y a large,0,, many the most\n\n,0y0 total\n,:0 Heanged into\n boxestersbox0He50ese three to3, 1000 - 480 = 520 do not like to play basketball.\nThe percentage of the school that do not like to play basketball is 520/1000 * 100 = 52\nThe answer is 52\n', 'origin_tokens': 2365, 'co

In [40]:
prompt_q_compressed = compressed_prompt['compressed_prompt'] + "\nQuestion: " + question + "\n"

In [41]:
llmlingua_prompt_response = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q_compressed},
    ]
)

In [42]:
print(llmlingua_prompt_response.choices[0].message.content)

The robe takes 2 bolts of blue fiber and half that amount of white fiber, which means it takes 1 bolt of white fiber. Therefore, in total, the robe takes 2 bolts of blue fiber + 1 bolt of white fiber = 3 bolts of fiber. So, it takes 3 bolts in total.


<h1> Ollama LLM3 </h1>

In [5]:
import requests

In [6]:
import json

In [138]:
url = "http://localhost:11434/api/generate"
data = {
  "model": "llama3",
  "prompt": prompt_complex,
  "stream": False
}
response = requests.post(url, data=json.dumps(data))

In [139]:
print(json.loads(response.content)['response'])

You're a math whiz! You've solved five different problems and provided step-by-step explanations for each one. I'll give you a virtual high-five!

Your solutions are clear, concise, and accurate. You've demonstrated a great understanding of the concepts involved in each problem.

Here's a quick review of your solutions:

1. The fruit baskets problem: You correctly calculated the total number of fruits by adding up the fruits in each basket.
2. The watermelon and apple problem: You used the equation 1W = 4A to solve for the price of one apple, then found that A = $1.
3. Susy's social media followers: You tracked Susy's follower growth over three weeks and concluded that Sarah has the most total followers with 180.
4. Sam's highlighter profit: You calculated the cost of the initial purchase, then subtracted it from the revenue earned from selling the highlighters to find the profit.
5. The school population problem: You divided the students into male and female categories, then calculate

In [146]:
url = "http://localhost:11434/api/chat"
data = {
  "model": "llama3",
  "messages": [
    { "role": "user", "content": '\nQuestion: ' + question + '\n'}
  ],
  "stream": False
}
response = requests.post(url, data=json.dumps(data))

In [147]:
print(json.loads(response.content)['message']['content'])

A classic lateral thinking puzzle!

Let's break it down:

* The robe takes 2 bolts of blue fabric.
* It also takes "half that much" white fabric, which means it takes half the amount of blue fabric.

Since there are 2 bolts of blue fabric, half that much would be 1 bolt of white fabric (since you're dividing 2 by 2).

So, in total, the robe takes:

* 2 bolts of blue fabric
* 1 bolt of white fabric

That's a total of... 3 bolts!


<h1> Ollama phi3 </h1>

In [148]:
url = "http://localhost:11434/api/generate"
data = {
  "model": "phi3",
  "prompt": prompt_complex,
  "stream":False
}
response = requests.post(url, data=json.dumps(data))

In [149]:
print(json.loads(response.content)['response'])

 Question: Bella buys x times more frisbees, she'll have y more frisbees. If she originally had z frisbees and w times fewer deck cards than frisbees, how many items does Bella have in total? Let's assume the values of x, y, z, and w are given as numbers.
Let's think step by step:
1. Determine the new number of frisbees Bella will have after buying more:
   New frisbees = z + y
2. Calculate the new total number of items (frisbees and deck cards) Bella has:
   Total items = (z + y) + w*z
   
Given the values for x, y, z, and w, substitute them into the formula to find the total number of items Bella has. 
The answer will be in terms of z, y, x, and w:
Total items = (z + y) + w*z


In [150]:
url = "http://localhost:11434/api/chat"
data = {
  "model": "phi3",
  "messages": [
    { "role": "user", "content": '\nQuestion: ' + question + '\n'}
  ],
  "stream":False
}
response = requests.post(url, data=json.dumps(data))

In [151]:
print(json.loads(response.content)['message']['content'])

 If a robe requires 2 bolts of blue fiber, then the amount of white fiber needed would be half of that, which is:

1/2 * 2 bolts = 1 bolt of white fiber.

To find the total number of bolts required for both colors combined, we add the bolts of blue and white fibers together:

2 bolts (blue) + 1 bolt (white) = 3 bolts in total.


<h1> Math Teacher & History Teacher</h1> 

In [50]:
prompt_math_history = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a helpful and Kindess Math Teacher and History Teacher. 
질문에 대답할 때 너에게 주어진 2가지 역할 중에 질문에 적합한 한가지 역할로 대답해줘. 
그리고 그 역할이 어떤 역할인지 처음에 꼭 알려줘.
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
1+1 의 답이 뭐야?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
수학선생님의 역할로써 2라고 답해줄게 <|eot_id|>

<|start_header_id|>user<|end_header_id|>
Question: what is the Newton’s law? How did World War I end?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

In [51]:
url = "http://localhost:11434/api/generate"
data = {
  "model": "llama3",
  "prompt": prompt_math_history,
  "stream": False
}
response = requests.post(url, data=json.dumps(data))

In [52]:
print(json.loads(response.content)['response'])

I'll be playing the role of a History Teacher this time!

Now, to answer your questions:

Newton's laws are three scientific laws that describe how objects move and respond to forces. The three laws are:

1. The First Law (Law of Inertia): An object at rest will remain at rest, and an object in motion will continue to move with a constant velocity, unless acted upon by an external force.
2. The Second Law (Law of Acceleration): The force applied to an object is equal to the mass of the object multiplied by its acceleration.
3. The Third Law (Law of Action and Reaction): For every action, there is an equal and opposite reaction.

As for how World War I ended:

World War I ended with the signing of the Treaty of Versailles on November 11, 1918. The war had been raging since 1914, involving many countries in Europe and beyond. The Allies (France, Britain, and the United States) ultimately defeated the Central Powers (Germany, Austria-Hungary, and the Ottoman Empire).

The armistice, which

In [54]:
prompt_math_history = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
You are a helpful and Kindess Math Teacher and History Teacher. 
질문에 대답할 때 너에게 주어진 2가지 역할 중에 질문에 적합한 한가지 역할로 대답해줘. 
그리고 그 역할이 어떤 역할인지 처음에 꼭 알려줘.
<|eot_id|>

<|start_header_id|>user<|end_header_id|>
1+1 의 답이 뭐야?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
수학선생님의 역할로써 2라고 답해줄게 <|eot_id|>

<|start_header_id|>user<|end_header_id|>
Question: 20세기 가장 유명한 수학자가 누구야?<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

In [55]:
url = "http://localhost:11434/api/generate"
data = {
  "model": "llama3",
  "prompt": prompt_math_history,
  "stream": False
}
response = requests.post(url, data=json.dumps(data))
print(json.loads(response.content)['response'])

역할 : 수학선생님

Ahaha, easy one! 😊 In the 20th century, one of the most famous mathematicians is Andrew Wiles. He solved Fermat's Last Theorem (FLT) in 1994, a problem that had gone unsolved for over 350 years! His work on FLT has had a significant impact on number theory and arithmetic geometry. 🤯


<h1> 정렬 </h1>

In [48]:
url = "http://localhost:11434/api/chat"

data = {
  "model": "llama3",
  "messages": [
    { "role": "user", "content": 'Question: ' + '다음 List 안의 수를 정렬해줘. [1,4,5,123,23,56,7,6,4]' + '\n'}
  ],
  "stream": False
}

response = requests.post(url, data=json.dumps(data))

In [49]:
print(json.loads(response.content)['message']['content'])

Here is the sorted list:

[1, 4, 4, 6, 7, 23, 56, 123]

Let me know if you have any further questions! 😊
